# Convolution Neural Network

Nous allons decouvrir un reseau qui nous permettra enfin d'obtenir 99% de reussite a la reconnaissance des chiffres !

In [ ]:
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import random

Dans le code qui suit nous allons charger les donnees MNIST, le code se chargera de telecharger les donnees pour vous, laissez lui un peu de temps.

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
s = ax.imshow(random.choice(mnist.test.images).reshape((28,28)), cmap = matplotlib.cm.binary)
plt.show()

Ci-dessus un exemple aleatoire de chiffre MNIST. Le code a peu d'interet, ne vous attardez pas dessus.

In [ ]:
sess = tf.InteractiveSession()

x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [ ]:
def generate_weights(x):
    return tf.Variable(tf.truncated_normal(x, stddev=0.1))

def generate_bias(x):
    return tf.Variable(tf.constant(0.1, shape=[x]))

Deux fonctions permettant d'initialiser les poids aleatoirement et les bias a une constante, pour eviter d'avoir un gradient a 0 ce qui empecherait l'apprentissage.

In [ ]:
def generate_conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def generate_max_pool(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

Deux fonctions, une pour creer un filtre de convolution et un pour creer un max pooling, si vous n'etes pas familier avec la notion de convolution l'image ci-dessous vous donnera l'intuition de ce que cela fait.

![convolution](https://developer.apple.com/library/content/documentation/Performance/Conceptual/vImage/Art/kernel_convolution.jpg)

Et pour ce qui est du max pooling:

![max pool](http://cs231n.github.io/assets/cnn/maxpool.jpeg)

Nous pouvons ainsi reduire la dimension de notre image tout en recuperant des informations "features" dans chaque pixel.

In [ ]:
W_conv1 = generate_weights([5, 5, 1, 32])
b_conv1 = generate_bias(32)

x_image = tf.reshape(x, [-1,28,28,1])

h_conv1 = tf.nn.relu(generate_conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = generate_max_pool(h_conv1)

Premier filtre de convolution, 32 feature maps et un max pool, on obtient alors des images de 14x14 pixels.

In [ ]:
W_conv2 = generate_weights([5, 5, 32, 64])
b_conv2 = generate_bias(64)

h_conv2 = tf.nn.relu(generate_conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = generate_max_pool(h_conv2)

A nouveau un filtre de convolution, 64 features maps cette fois et suivi d'un max pooling, nous obtenons donc 64 images de 7x7 pixels.

In [ ]:
W_fc1 = generate_weights([7 * 7 * 64, 1024])
b_fc1 = generate_bias(1024)

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

On applatit nos 64 images en un seul vecteur que l'on passe a un reseau de neurone type multilayer perceptron comme nous avons vu dans le tutoriel 2.


In [ ]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

On ajoute un **dropout**, c'est a dire que l'on va volontairement jetter le resultat de certains neurones et ce pour eviter que notre modele devienne trop performant sur les donnees d'entrainement, nous voulons aussi qu'il fonctionne bien avec les donnees de validation!

In [ ]:
W_fc2 = generate_weights([1024, 10])
b_fc2 = generate_bias(10)

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

On finalise le modele pour obtenir 10 sorties, une pour chaque chiffre.

In [ ]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_conv, y_))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.global_variables_initializer())
for i in range(20000):
    batch = mnist.train.next_batch(50)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})



In [ ]:
taux_reussite = accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0})

print("Taux de reussite : " + str(taux_reussite))

Prennez le temps de bien etudier ce code et ce reseau, c'est un des plus important dans la reconnaissance d'image, il est peu commente car vous etes suppose avoir une bonne comprehension des fondements de tensorflow et des reseaux de neurones denses.